In [1]:
from sklearn import preprocessing
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as mp
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('sentence-transformers/sentence-t5-base')

In [3]:
training_data_path = 'training.xlsx'
df = pd.read_excel(training_data_path)

In [4]:
#Removing line breaks in input column

def remove(s):
    if isinstance(s, str):
        return ' '.join(s.splitlines())
    else:
        return s

In [5]:
#Embeddings & dropping text solutions

vectorized_df= pd.DataFrame(df['input'])
vectorized_df['input'] = vectorized_df['input'].apply(remove)
vectorized_df['EmbeddingsLM'] = vectorized_df['input'].apply(lambda x:model.encode(str(x)))

#splitting into 768 different columns
vectorized_df['EmbeddingsLM'] = vectorized_df['EmbeddingsLM'].apply(lambda x: x.tolist())
num_columns = len(vectorized_df['EmbeddingsLM'][0])

for i in range(num_columns):
    col_name = "Embedding_"+str(i)
    vectorized_df[col_name] = vectorized_df['EmbeddingsLM'].apply(lambda x: x[i])
 

In [6]:
vectorized_df.drop(columns=['input','EmbeddingsLM'], inplace=True)
vectorized_df['output'] = df['output']

In [7]:
#Class labelling
# 0-2 incorrect
# 2-3 part correct
# 3-5 correct

def classes(marks):
    if (0<=marks<2):
        return 'incorrect'
    elif (2<=marks<3):
        return 'partially correct'
    elif (3<=marks<=5):
        return 'correct'

vectorized_df['class'] = vectorized_df['output'].apply(classes)

#label encoding the classes
# 0 - incorrect
# 1 - partially correct
# 2 - correct

def labelenc(cls):
    if(cls=='incorrect'):
        return 0
    elif (cls=='partially correct'):
        return 1
    elif (cls=='correct'):
        return 2

numeric_df = vectorized_df.copy()
numeric_df['class'] = numeric_df['class'].apply(labelenc)

vectorized_df.to_excel('vectorized_dataset.xlsx', index=False)
numeric_df.to_excel('numeric_dataset.xlsx', index=False)